***
# IT575 - Computational Shape Modeling
## Mesh Deformation using Skinning - November 11, 2024
### Group no. $\underline{\hspace{1in}}$
### Member 1 Name: $\underline{\hspace{1in}Vraj Thakkar}$, Member 1 ID: $\underline{\hspace{1in}202103052}$
### Member 2 Name: $\underline{\hspace{1in}Dhruv Shah}$, Member 2 ID: $\underline{\hspace{1in}202103017}$
***

In [1]:
# Import IGL and required functions
import igl
import numpy as np
from meshplot import plot, subplot, interact
import meshplot as mp
import matplotlib.pyplot as plt
from mat4py import loadmat

### Run the code below to plot the meshes
#### Help for Meshplot: https://skoch9.github.io/meshplot/tutorial/ 
#### Help for Libigl in Python: https://libigl.github.io/libigl-python-bindings/

In [2]:
# Uncomment the one that you wish to plot
###### Cylinder #############
dt = igl.read_off("cylinder.off")
h = [4800,4801] # Handles for Cylinder
#############################
###### Cactus ###############
# dt = igl.read_off("cactus.off")
# h = [2233,4558,3020,1796] # Handles for Cactus
#############################
v = dt[0]
f = dt[1]
p=plot(v, f,return_plot=True)
p.add_points(v[h,:], c=np.array([1.0,0.0,0.0]), shading={"point_size": 1}) # point_size 1 for Cylinder, 0.05 for Cactus

Error: bad line (0)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Invalid color array given! Supported are numpy arrays. <class 'numpy.ndarray'>


1

In [3]:
#######################################################################################
# Skinning Weights-------------
# Input: h - array containing handle indices
#        v - nov x 3 array containing vertex coordinates in each row. nov is the number of vertices in the 
#            mesh.
#        f - nof x 3 array containing indices of the array 'v', each row of f points to vertices 
#            of a triangle in the mesh
#     wttype - If wttype = 'InvDist' it will compute Inverse Euclidean distance based weights. Else, it will 
#            load pre-computed BBW weights for either Cylinder or Cactus mesh with handles h=[4800, 4801]
#              (bottom, top) for Cylinder, h=[2233,4558,3020, 1796],i.e, bottom, top, left and right tips
#              for Cactus.
# Output: w - nov x noh matrix, nov is the number of vertices, noh is the number of handles. Each column 
#             corresponds to the weight associated with one handle over the entire mesh.
#######################################################################################
def skinningweights(h,v,f,wttype):
    noh = len(h)
    nov = v.shape[0]
    W = np.zeros((nov,noh))
    if wttype == 'InvDist':
        for i in np.arange(noh):
            W[:,i] = 1/(1+np.linalg.norm(v - v[h[i],:],axis=1))
        for i in np.arange(nov):
            W[i,:] = W[i,:]/(np.sum(W[i,:]))
    else:
        ############################################################
        # Else use Bounded-Biharmonic Weights. These weights have been pre-computed and saved for Cylinder
        # and Cactus meshes in the files: BBW_cylinder.mat and BBW_cactus.mat. These MATLAB files can be 
        # loaded in python using the function 'loadmat'.
        # Note that for Cylinder the handles are H=[4800, 4801](bottom, top), for Cactus the handles are 
        #H = [2233,4558,3020, 1796],i.e, bottom, top, left and right tips.
        ############################################################
        data = loadmat('BBW_cylinder.mat') # Uncomment to load BBW weights for Cylinder mesh,
        #data = loadmat('BBW_cactus.mat') # Uncomment to load BBW weights for Cactus mesh,
        H = np.array(data['H'])
        W = np.array(data['w'])
    return W                 

#### Q.1. Display the mesh with vertex color defined by Skinning weights. Try out Cylinder and Cactus mesh with Inverse Distance and Bounded Biharmonic Weights, and make your observations

In [4]:
# Uncomment the one that you wish to plot
###### Cylinder #############
dt = igl.read_off("cylinder.off")
h = [4800,4801] # Handles for Cylinder
#############################
###### Cactus ###############
# dt = igl.read_off("cactus.off")
# h = [2233,4558,3020, 1796] # Handles for Cactus
#############################

### Skinning Weights --------------
w = skinningweights(h,v,f,'InvDist') # Uncomment for Inverse Distance weights
#w = skinningweights(h,v,f,'BBW') # Uncomment for BBW weights
######################################

### Display meshes------------------------------
p=plot(v,f,c=w[:,0]) # Color is given as per the weights of the first handle. 
p.add_points(v[h,:], c=np.array([1.0,0.0,0.0]), shading={"point_size": 1}) # point_size 1 for Cylinder, 0.05 for Cactus
#################################################

Error: bad line (0)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Invalid color array given! Supported are numpy arrays. <class 'numpy.ndarray'>


1

#### Q.2 In the following cell, there are some basic operations you will require in terms of rotation matrices and quaternions. Complete the functions as per the description given for each function

In [20]:
#####################################################################
# Convert axis-angle representation to 3 x 3 Rotation matrix
# Rodrigues formula {R = I + sin(theta)*[n]x+ (1-cos(theta)) (Nx x Nx )}, where x denotes Vector cross product.
# Input: 
#     theta: Angle of rotation. Scalar
#     n    : Axis of rotation. 3x1 vector
# Output: 
#     R    : 3x3 Rotation matrix 
#####################################################################
def RodriguesRotation(theta,n):
    # Use Rodrigues' rotation formula to obtain R from axis angle representation.
    # Write your code here
    I = np.eye(3)
    nx = np.zeros((3,3))
    nx[0,1] = -n[2]
    nx[0,2] = n[1]
    nx[1,0] = n[2]
    nx[1,2] = -n[0]
    nx[2,0] = -n[1]
    nx[2,1] = n[0]
    R = I + np.sin(theta)*nx + (1-np.cos(theta))*np.matmul(nx,nx)
    return R
#####################################################################
# Multiplication of two quaternions.    
# Input: q1, q2 - (4,1) vectors to be multiplied
# Output: product of the two quaternions
#####################################################################
def qmult(q1, q2):
    # Compute quaternion product.
    real= q1[0]*q2[0] - np.dot(q1[1:4],q2[1:4])
    imag = q1[0]*q2[1:4] + q2[0]*q1[1:4] + np.cross(q1[1:4],q2[1:4])
    q = np.zeros(4)
    q[0] = real
    q[1:4] = imag
    return q
    
#####################################################################
# Quaternion transformation q application to a vector v
# Input: q - Unit quaternion that encodes the transformation to be applied
#        v - the 3 x 1 vertex coordinates on which the transformation encoded in q has to be applied.
# Output: defv = q . v . q*. is a 3 x 1 vector of coordinates of deformed vertex.
#####################################################################
def myQuatonv(q,v):
    # compute defv = q*v*q^*, and return the vector part as defv.
    qv = np.zeros(4)
    qv[1:4] = v
    qc = np.zeros(4)
    qc[0]= q[0]
    qc[1:4] = -q[1:4]
    defv = qmult(qmult(q,qv),qc)
    return defv[1:4]

######################################################################
# Unit Quaternion log
# Input: q - Unit quaternion
# Ouput: logq - Log of the unit quaternion of the form [0;theta * axis] - 4 x 1 vector
######################################################################
def myQlog(q):
    # Compute theta and axis n. Return [0;theta * axis]
    q = q/np.linalg.norm(q)
    theta = np.arccos(q[0])
    if np.isclose(theta, 0) or np.isclose(theta, np.pi):
        n = np.zeros(3)
    else:
        n = q[1:4] / np.sin(theta)
    logq = np.zeros(4)
    logq[0] = 0
    logq[1:4] = theta*n
    return logq

######################################################################
# Quaternion Exponential of a quaternion of the form [0;theta*axis]
# Input: q - Quaternion of the form [0;theta *axis] - 4 x 1 vector
# Ouput: expq - Exp of the quaternion, of the form [cos(theta);sin(theta)*axis] - 4 x 1 vector. Should be a
#               unit quaternion.
######################################################################
def myQexp(logq):
    # Compute theta and axis, and then compute the exp.
    logq = np.array(logq)
    theta = np.linalg.norm(logq[1:4])
    if np.isclose(theta, 0):
        expq = np.zeros(4)
        expq[0] = 1
    else:
        n = logq[1:4] / theta
        expq = np.zeros(4)
        expq[0] = np.cos(theta)
        expq[1:4] = np.sin(theta) * n
    return expq
## Q.2. The following code implements blending of matrices, followed by applying the blended matrices on the original mesh vertices, and returns the deformed vertices. It linearly blends transformations or quaternions, as per the user input. Complete the code.

#####################################################################
# Quaternion to 3D rotation
# Input: q - Input unit quaternion as (4x1) vector that is to be converted into 3 x 3 matrix.
# Output: R - 3 x 3 rotation matrix.
#####################################################################
def myQtoSO3(q):
    T = np.zeros((3,3))
    sintby2 = np.linalg.norm(q[1:len(q)])
    costby2 = q[0]
    theta = 2*np.arctan2(sintby2,costby2)
    if np.abs(sintby2) < 1e-14:
        eta = q[1:len(q)]
    else:
        eta = q[1:len(q)]/sintby2
    R = RodriguesRotation(theta,eta)
    return R

#### Q.3. The following code implements blending of rotations, using matrices or quaternions, followed by applying the blended matrices/quaternions on the original mesh vertices, and returns the deformed vertices. It linearly blends transformations or quaternions, as per the user input. Complete the code.

In [37]:
#############################################
# Blends the weights and Handle transformations to obtain per vertex transformation, 
# which is then applied to vertices of the input mesh to obtain the deformed Mesh
# Input: w - nov(number of vert.) x noh(number of handles) matrix, each column containing 
#            weights of corresponding handles
#        H - (noh,) array. Indices of handle vertices from the vertices (v) array
#        v - (nov,3) array containing vertex coordinates in each row
#      eta - is expected as a 3 x noh matrix, each column containing the axis of rotation transformation
#            to be applied to the corresponding handle.
#     theta - is expected as a (noh,) array containing the angles of rotations in radians to be applied to 
#            corresponding handles.
#       typ - If typ = 'LBS', the function implements Linear Blend Skinning, if typ = 'LERP' it implements 
#             Linear Blending of Quaternions, else it implements SLERP.
# Output: DefV - is a nov x 3 array containing coordinates of the deformed mesh. We assume same connectivity 
#             as that of the input mesh.
#############################################
def BlendWT(w,H,v,eta,theta,typ):
    nov = w.shape[0]
    noh = len(H)
    Tv = np.zeros((3,3,nov))
    DefV = np.zeros((nov,3))
            
    if typ=='LBS':
        T = np.zeros((3,3,noh))
        for i in np.arange(noh):
            # Convert axis-angle representation to matrix representation.
            # Fill in your code below.
            T[:,:,i] = RodriguesRotation(theta[i],eta[:,i])
        for i in np.arange(nov):
            # Perform LBS below
            # Fill in your code below.
            Tv[:,:,i] = np.sum(w[i,:]*T,axis=2)
            # Apply the blended transformation to each vertex of original mesh.
            # Fill in your code below.
            DefV[i,:] =  Tv[:,:,i]@v[i,:]
            
    elif typ=='LERP': #Linear Blending of Unit Quaternions
        q = np.zeros((4,noh))
        for i in np.arange(noh):
            q[:,i] = np.append([np.cos(theta[i]/2)],[np.sin(theta[i]/2)*eta[:,i]])
                
        qv = np.zeros((4,nov))
        for i in np.arange(nov):
            # Perform linear blending of quaternions
            # Fill in your code below.
            #qv[:,i] = np.sum(w[i,:]*q,axis=1)
            qv[:,i] = w[i,:]@q.T
            #Below segment normalizes the quaternion to obtain unit quaternions.
            if np.linalg.norm(qv[:,i]) < 1e-15:
                qv[:,i] = qv[:,i]
            else:
                qv[:,i] = qv[:,i]/np.linalg.norm(qv[:,i])
            #Apply transformation encoded as quaternion to corresponding vertex of original mesh
            # to obtain Deformed mesh vertex. Fill in your code below.
            #Tv = myQtoSO3(qv[:,i])
            #DefV[i,:] = Tv@v[i,:]
            DefV[i,:] = myQuatonv(qv[:,i],v[i,:])
    else: # SLERP
        q = np.zeros((4,noh))
        for i in np.arange(noh):
            q[:,i] = np.append([np.cos(theta[i]/2)],[np.sin(theta[i]/2)*eta[:,i]])
        qv = np.zeros((4,nov))
        for i in np.arange(nov):
            # Perform SLERP
            # Fill in your code below
            qcm = np.array([1,0,0,0])
            for j in np.arange(noh):
                prem = qcm
                prem[1:4] = -1*qcm[1:4]
                q1q2 = qmult(prem,q[:,j])
                qc = myQexp(w[i,j]*myQlog(q1q2))
                qcm = qmult(qcm,qc)
            qv[:,i] = qcm/np.linalg.norm(qcm)
            #Apply transformation encoded as quaternion to corresponding vertex of original mesh
            # to obtain Deformed mesh vertex. Fill in your code below.
            # Tv = myQtoSO3(qv[:,i])
            # DefV[i,:] = Tv@v[i,:]
            DefV[i,:] = myQuatonv(qv[:,i],v[i,:])
    return DefV
    

#### Q.4 . Try out some deformations using Skinning on Cylinder Mesh, with LBS, LERP and SLERP

In [40]:
#### Read Mesh #########
dt = igl.read_off("cylinder.off")
v = dt[0]
f = dt[1]
plot(v,f)
#################################
#### Handles and Handle deformations ########
h = np.array([4800,4801])
eta = np.array([[0,0,1],[0,0,1]]).T
theta = np.array([0,np.pi])
#################################
##### Skinning Weights##########
w = skinningweights(h,v,f,'InvDist') # Uncomment for InvDist weights
#w = skinningweights(h,v,f,'BBW') # Uncomment for BBW weights
#defv = BlendWT(w,h,v,eta,theta,'LBS')  # Uncomment for LBS blending
#defv = BlendWT(w,h,v,eta,theta,'LERP') # Uncomment for LERP
defv = BlendWT(w,h,v,eta,theta,'SLERP') # Uncomment for SLERP
plot(defv,f)

Error: bad line (0)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-2.086162…

#### Q.5. Try out some deformations using Skinning on Cactus Mesh using LBS, LERP and SLERP

In [39]:
#### Read Mesh #####################
dt = igl.read_off("cactus.off")
v = dt[0]
f = dt[1]
####################################

#### Handles and Handle deformations ########
h = np.array([2233,4558,3020,1796])
eta = np.array([[0,1,0],[1,0,0],[1,0,0],[1,0,0]]).T
theta = np.array([0,np.pi/10,0,np.pi/12])
################################################

##### Skinning Weights##########
w = skinningweights(h,v,f,'InvDist') # Uncomment for InvDist weights
#w = skinningweights(h,v,f,'BBW') # Uncomment for BBW weights. Remember to change the file to 
                                  # BBW_Cactus if using BBW weights.

#defv = BlendWT(w,h,v,eta,theta,'LBS')  # Uncomment for LBS blending
#defv = BlendWT(w,h,v,eta,theta,'LERP') # Uncomment for LERP
defv = BlendWT(w,h,v,eta,theta,'SLERP') # Uncomment for SLERP

plot(defv,f)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5000501…